<a href="https://colab.research.google.com/github/AlySalem94/UIC-SWMM_BWDF/blob/main/BWDF_Online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Multivariate LSTM Model for Short-term Water Demand Forecasting**

***Battle of Water Demand Forecast - CCWI 2024***

**Team Name: UIC-SWIM**

**Team Members:**
1.   [Aly Salem](https://www.linkedin.com/in/aly-salem-70b97813b/) - Ph.D. Candidate, Department of Civil, Materials, and Environmental Engineering, The University of Illinois Chicago - asalem22@uic.edu
2.   [Ahmed Abokifa](https://www.linkedin.com/in/ahmed-abokifa/) - Assistant Professor, Department of Civil, Materials, and Environmental Engineering, The University of Illinois Chicago - abokifa@uic.edu

# Update Pandas

In [1]:
!pip install pandas==2.2.0

# Import Necessary Packages

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import plotly.io as pio
import time
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

# Define Inputs and Training Parameters

In [3]:
# Define DMA name
DMA_name = 'A'

# Define file paths and date format
demand_file = 'Inflow.xlsx'
weather_file = 'Weather.xlsx'
holidays_file = 'Holidays.xlsx'

# Define training parameters
input_sequence = 168 # in hours
output_sequence = 168 # in hours
n_epochs = 1000
batch_size = 100
learning_rate = 0.001
hidden_size = [10,128]
num_layers = 2

colab = False
filling_days = 10

# Load and Prepare Input Data

In [4]:
!git clone https://github.com/AlySalem94/UIC-SWMM_BWDF.git

data_dir = os.path.join('/content/UIC-SWMM_BWDF')

# Load data from Excel files
demand_df = pd.read_excel(f"{data_dir}/{demand_file}", parse_dates=[0], index_col=0)
weather_df = pd.read_excel(f"{data_dir}/{weather_file}", parse_dates=[0], index_col=0)
holiday_df = pd.read_excel(f"{data_dir}/{holidays_file}", parse_dates=[0], index_col=0)

# Assign DMA names and meteorological data to demand_df and weather_df
demand_df.columns = [chr(ord('A') + i) for i in range(len(demand_df.columns))]
weather_df.columns = ['Rainfall depth', 'Air temperature', 'Air humidity', 'Windspeed']

# Convert index to datetime format
date_format = '%d/%m/%Y %H:%M'
start_date = '2021-01-01 00:00:00'
end_date = '2022-07-24 23:00:00'
demand_df.index = pd.to_datetime(demand_df.index, format=date_format)
weather_df.index = pd.to_datetime(weather_df.index, format=date_format)
holiday_df.index = pd.to_datetime(holiday_df.index, format=date_format)

# Merge dataframes on the date index
data_df = pd.merge(demand_df[DMA_name], weather_df, left_index=True, right_index=True, how='outer')
data_df = pd.merge(data_df, holiday_df, left_index=True, right_index=True, how='outer')

# Select data within the specified date range
data_df = data_df.loc[start_date:end_date]

Cloning into 'UIC-SWMM_BWDF'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 1.71 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (2/2), done.


# Create a Directory for Results

In [5]:
# Get the the run time
dt_string = datetime.now().strftime("%m-%d_%H-%M-%S")

# Check if the 'Results' folder exists, and create it if not
results_folder = os.path.join('/content/UIC-SWMM_BWDF', 'Results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Create a folder with the DMA name and run time
os.makedirs(os.path.join('/content/UIC-SWMM_BWDF', 'Results', f"{DMA_name}_{dt_string}"))

results_dir = os.path.join('/content/UIC-SWMM_BWDF', 'Results', f"{DMA_name}_{dt_string}")


# Manage Summer and Standard time

In [6]:
# Handle standard time (2 am is repeated at 10/31/2021 & 10/30/2022)
data_df = data_df[~data_df.index.duplicated(keep='first')]

# Handle summer time (2 am is missed hour at 3/28/2021 & 3/27/2022)
summer_index = pd.to_datetime(['2021-03-28 02:00:00', '2022-03-27 02:00:00'])
summer_df = pd.DataFrame(index=summer_index)
data_df = pd.concat([data_df, summer_df])

# Sort the index to maintain order
data_df.sort_index(inplace=True)
data_df_original = data_df.copy()

# Save Run Details to a Results File

In [7]:
with open(f"{results_dir}/Results.txt", "a") as f:
    f.write(f"DMA_name = {DMA_name}\n")
    f.write(f"input_sequence = {input_sequence}\n")
    f.write(f"output_sequence = {output_sequence}\n")
    f.write(f"n_epochs = {n_epochs}\n")
    f.write(f"batch_size = {batch_size}\n")
    f.write(f"learning_rate = {learning_rate}\n\n")
    f.write(f"hidden_size = {hidden_size}\n")
    f.write(f"num_layers = {num_layers}\n\n")
    f.write(f"demand_file = '{demand_file}'\n")
    f.write(f"weather_file = '{weather_file}'\n")
    f.write(f"date_format = '{date_format}'\n\n")
    f.write(f"start_date = '{start_date}'\n")
    f.write(f"end_date = '{end_date}'\n\n")
    f.write("_____________________________________________________________\n")


# Handle Missing Values Using Wieghted Moving Mean

In [8]:
# Loop through each input column
for column in data_df.columns.values:
    # Create a date index from start to end dates
    date_index = pd.date_range(start=data_df.index[0].date(), end=data_df.index[-1].date(), freq='D')

    # Create an hourly DataFrame with 24 columns representing hours in each day
    hourly_df = pd.DataFrame(index=date_index, columns=range(0, 24))
    hourly_df = hourly_df.apply(pd.to_numeric, errors='coerce')

    # Fill the hourly DataFrame with values from the original data_df
    for date in hourly_df.index:
        for hour in range(0, 24):
            hourly_df.loc[date, hour] = data_df.loc[date + pd.Timedelta(hours=hour), column]

    for hour in hourly_df.columns:
        missing_dates = hourly_df[hour].isnull()

        for missing_index in hourly_df.index[missing_dates]:
            dates_upto_missing_date = pd.date_range(start=pd.to_datetime(start_date).replace(hour=missing_index.hour), end=missing_index, freq=pd.Timedelta(hours=24))

            previous_values = hourly_df.loc[dates_upto_missing_date[-(filling_days)-1:-1], hour]

            weights = (np.array([*range(1, len(previous_values)+1)])/np.array([*range(1, len(previous_values)+1)]).sum())

            hourly_df.loc[missing_index, hour] = (previous_values*weights).sum()

    # Reconstrcut the data_df with the filled values
    data_df[column] = hourly_df.values.reshape(-1)

start_date = datetime(2021, 1, 1)
date_values = [start_date + timedelta(hours=i) for i in range(len(data_df_original))]


trace_original = go.Scatter(x=date_values, y=data_df_original.loc[start_date:date_values[-1]].values[:,0], mode='lines', name='Original')

trace_filled = go.Scatter(x=date_values, y=data_df.loc[start_date:date_values[-1]].values[:,0], mode='lines', name='Original')

layout = go.Layout(title='', legend=dict(x=0, y=1, traceorder='normal', orientation='h'))

fig = go.Figure(data=[trace_filled, trace_original], layout=layout)

fig.show()

KeyError: Timestamp('2022-01-08 00:00:00', freq='D')

# Extract/Add Hour, Day, and Weekday Features

In [ ]:
data_df['hour'] = data_df.index.hour
data_df['day_of_year'] = data_df.index.dayofyear
data_df['weekday'] = data_df.index.weekday

# Standardize Input/Output, Apply PCA and Data Transformation

In [ ]:
X_input = data_df
y_input = np.array(data_df[DMA_name])
X_input.shape, y_input.shape

mm = MinMaxScaler()
ss = StandardScaler()

X_std = ss.fit_transform(X_input)
y_std = mm.fit_transform(y_input.reshape(-1, 1))
# y_std = y_input.reshape(-1, 1)

pca = PCA(4)
pca.fit(X_std)

X_std = pca.transform(X_std)

# Split Data into Input/Output Sequences

In [ ]:
# split a multivariate sequence into samples
def split_sequences(input_sequence, output_sequence, n_steps_in, n_steps_out):
    X, y = list(), list() # instantiate X and y
    for i in range(len(input_sequence)):
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(input_sequence): break
        # gather input and output of the pattern
        seq_x, seq_y = input_sequence[i:end_ix], output_sequence[end_ix:out_end_ix, -1]
        X.append(seq_x), y.append(seq_y)
    return np.array(X), np.array(y)

X, y = split_sequences(X_std, y_std, input_sequence, output_sequence)
print(X.shape, y.shape)

# Convert Data to Tensors

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

total_samples = len(X)
train_test_cutoff = X.shape[0]-input_sequence

X_train, X_test = torch.Tensor(X[0:train_test_cutoff]).to(device), torch.Tensor(X[train_test_cutoff:])[-1].reshape(1,input_sequence,-1).to(device)
y_train, y_test = torch.Tensor(y[0:train_test_cutoff]).to(device), torch.Tensor(y[train_test_cutoff:])[-1].reshape(1,-1).to(device)

print("Training Shape:", X_train.shape, y_train.shape)
print("Testing Shape:", X_test.shape, y_test.shape)

# Define The LSTM Model and The Loss Function

In [ ]:
print(device)

In [ ]:
# The LSTM model
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size[0] # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size[0],
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size[0], hidden_size[1]) # fully connected
        self.fc_2 = nn.Linear(hidden_size[1], num_classes) # fully connected last layer
        self.relu = nn.ReLU()
        self.to(device)

    def forward(self,x):
        x = x.to(device)
        # hidden state
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # cell state
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn[-1,:,:].view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

# The Loss Function
class PI(torch.nn.Module):
    def __init__(self):
        super(PI, self).__init__()

    def forward(self, predicted_values, true_values):

        true_values = true_values.reshape(-1,7,24)

        predicted_values = predicted_values.reshape(-1,7,24)

        absolute_diff = torch.abs(true_values - predicted_values)

        PI1 = absolute_diff.mean(dim=2).mean(dim=1)
        PI2 = absolute_diff[:, 0, :].max(dim=1).values
        PI3 = absolute_diff[:, 1:, :].mean(dim=2).mean(dim=1)

        return PI1.mean() + PI2.mean() + PI3.mean()

# Define The Training Loop and The Performance Indicators (PI)

In [ ]:
# Training loop
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train,
                  X_test, y_test, batch_size):

    for epoch in range(n_epochs):
        lstm.train()

        total_train_loss = 0.0  # Accumulate training loss over all mini-batches

        # Iterate over mini-batches
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            outputs = lstm.forward(X_batch)  # forward pass
            optimiser.zero_grad()  # calculate the gradient, manually setting to 0
            loss = loss_fn(outputs, y_batch)
            loss.backward()  # calculates the loss of the loss function
            optimiser.step()  # improve from loss, i.e backprop

            total_train_loss += loss.item()  # Accumulate the loss

        average_train_loss = total_train_loss / (len(X_train) / batch_size)

        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)

        if epoch % 100 == 0:
            # Print average time over the last 100 epochs
            print("Epoch: %d, train loss/mini patch: %1.5f, test loss for W29/2022: %1.5f" % (epoch, average_train_loss, test_loss.item()))

            with open(f"{results_dir}/Results.txt", "a") as f:
                f.write("Epoch: %d, train loss/mini patch: %1.5f, test loss for W29/2022: %1.5f \n" % (epoch, average_train_loss, test_loss.item()))

# The Performance Indicators (PI)
def PI_test(predicted_values, true_values):

    true_values = torch.from_numpy(true_values)
    predicted_values = torch.from_numpy(predicted_values)

    true_values = true_values.reshape(-1,7,24)

    predicted_values = predicted_values.reshape(-1,7,24)

    absolute_diff = torch.abs(true_values - predicted_values)

    PI1 = absolute_diff.mean(dim=2).mean(dim=1)
    PI2 = absolute_diff[:, 0, :].max(dim=1).values
    PI3 = absolute_diff[:, 1:, :].mean(dim=2).mean(dim=1)

    return PI1.mean(), PI2.mean(), PI3.mean()


# Model Training/Testing

In [ ]:
# Create an instance of the LSTM model
lstm = LSTM(output_sequence, np.size(X,2), hidden_size, num_layers)
loss_fn = PI()
optimiser = torch.optim.Adam(lstm.parameters(), lr=learning_rate)


# Train the LSTM model
t1 = time.time()
print(f'run: {DMA_name}_{dt_string} started at: {dt_string}')
training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train, X_test, y_test, batch_size)
t2 = time.time()
print(f'run: {DMA_name}_{dt_string} ended in: {(t2-t1):.2f} sec.')
with open(f"{results_dir}/Results.txt", "a") as f:
    f.writelines('_____________________________________________________________'+ '\n')
    f.write(f"run: {dt_string}, ended in: {(t2-t1):.2f} sec.\n")

# Save the trained model
path = os.path.join(results_dir, 'LSTM_model')
torch.save(lstm.state_dict(), path)

# # Load the model if already trained
# path = os.path.join(os.getcwd(), 'results', 'A_01-31_14-14-02', 'LSTM_model')
# model_data = torch.load(path, map_location=device)
# lstm.load_state_dict(model_data)

# Make predictions for W29/2022
lstm.eval()
predicted_test = lstm(X_test).to('cpu').detach().numpy()
predicted_test = mm.inverse_transform(predicted_test)
actual_test = mm.inverse_transform(y_test.to('cpu'))

# Calculate the PI
PI1, PI2, PI3 = PI_test(actual_test, predicted_test)

print(f'PI1 : {PI1}, PI2 : {PI2}, PI3 : {PI3}')
with open(f"{results_dir}/Results.txt", "a") as f:
    f.writelines('_____________________________________________________________'+ '\n')
    f.write(f"PI1 : {PI1}, PI2 : {PI2}, PI3 : {PI3}\n")

# Plot Model Results

In [ ]:
# Plot actual vs. predicted values for W29/2022
start_date = datetime(2021, 1, 1) + timedelta(hours=input_sequence)
date_values = [start_date + timedelta(hours=i) for i in range(len(y))]

start_date = date_values[-1]
date_values = [start_date + timedelta(hours=i) for i in range(output_sequence)]

trace_actual = go.Scatter(x=date_values, y=actual_test.reshape(-1), mode='lines', name='Actual Data')
trace_predicted = go.Scatter(x=date_values, y=predicted_test.reshape(-1), mode='lines', name='Predicted Data')

layout = go.Layout(title='WDF for W29/2022', legend=dict(x=0, y=1, traceorder='normal', orientation='h'))
fig = go.Figure(data=[trace_actual, trace_predicted], layout=layout)


fig.show()


In [ ]:
X_forecast = torch.Tensor(pca.transform(ss.transform(X_input[-input_sequence-output_sequence:-output_sequence].values)).reshape(1,input_sequence,-1)).to(device)

# Make predictions for W30/2022
lstm.eval()
predicted_forecast = lstm(X_forecast).to('cpu').detach().numpy()
predicted_forecast = mm.inverse_transform(predicted_forecast)


# Plot forecast values for W30/2022
start_date = datetime(2021, 1, 1) + timedelta(hours=input_sequence)
date_values = [start_date + timedelta(hours=i) for i in range(len(y)+output_sequence)]
start_date = date_values[-1]
date_values = [start_date + timedelta(hours=i) for i in range(output_sequence)]

trace_predicted = go.Scatter(x=date_values, y=predicted_forecast.reshape(-1), mode='lines', name='Predicted Data')

layout = go.Layout(title='WDF for W30/2022', legend=dict(x=0, y=1, traceorder='normal', orientation='h'))
fig = go.Figure(data=[trace_predicted], layout=layout)


fig.show()

In [ ]:
X_forecast = torch.Tensor(pca.transform(ss.transform(X_input[-input_sequence:].values)).reshape(1,(input_sequence),-1)).to(device)

# Make predictions for W30/2022
lstm.eval()
predicted_forecast = lstm(X_forecast).to('cpu').detach().numpy()
predicted_forecast = mm.inverse_transform(predicted_forecast)


# Plot forecast values for W30/2022
start_date = datetime(2021, 1, 1) + timedelta(hours=input_sequence)
date_values = [start_date + timedelta(hours=i) for i in range(len(y)+output_sequence)]
start_date = date_values[-1]
date_values = [start_date + timedelta(hours=i) for i in range(output_sequence)]

trace_predicted = go.Scatter(x=date_values, y=predicted_forecast.reshape(-1), mode='lines', name='Predicted Data')

layout = go.Layout(title='WDF for W30/2022', legend=dict(x=0, y=1, traceorder='normal', orientation='h'))
fig = go.Figure(data=[trace_predicted], layout=layout)


fig.show()

In [ ]:
torch.Tensor(pca.transform(ss.transform(X_input[-input_sequence:].values)).reshape(1,(input_sequence),-1)).to(device).shape

In [ ]:
torch.Tensor(pca.transform(ss.transform(X_input[-input_sequence-output_sequence:-output_sequence].values)).reshape(1,input_sequence,-1)).to(device).shape

In [ ]:
X_input[-input_sequence:].values